In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import umap
import umap.plot

from sklearn.neighbors import LocalOutlierFactor #contamination
from sklearn.cluster import DBSCAN #eps=0.5, min_samples=5,
from sklearn.decomposition import PCA, FastICA  #n_components
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, classification_report

# Add parent folder to syspath to include local util functions
sys.path.insert(0, os.path.abspath('..'))
from utils.load_data_utils import load_data
from utils.plot_utils import plot_embedding

In [ ]:
data_dir = "../../data/"
plots_dir = "../../plots/"
# Options: "IterativeImputeBayesed", "IterativeImputeRFed", "IterativeImputeKNNed", "mean_filled", "median_filled"
data_dict = load_data("yeojohnsoned_z/iterative-filled")
data_dict.keys()

In [ ]:
# Get partitioned data:
train_data, test_data = data_dict["train_data"], data_dict["test_data"]
train_pod, test_pod = data_dict["train_pod"], data_dict["test_pod"]
train_pocd, test_pocd = data_dict["train_pocd"], data_dict["test_pocd"]

In [ ]:
# Absolute difference of data in percent (just for checking differences in distributions):
diffs = np.round(np.abs(test_data.mean(axis=0) - train_data.mean(axis=0)) / data_dict["all_data"].mean(axis=0), 2)
diffs

In [ ]:
train_data.mean()

## Decomposition / Dimensionality Reduction

### Independent Component Analysis (ICA)

In [ ]:
# reconstruct the Pandas df with labels
df = pd.DataFrame(data_dict["all_data"], columns=data_dict["data_names"])
# add back POD and POCD columns
df['POD'] = data_dict["all_pod"]
df['POCD'] = data_dict["all_pocd"]
# separate data by sex:
males, females = [x for _, x in df.groupby(df['sex'] == 1.0)]

In [ ]:
def train_ica(data_frame):
    # as ICA is stochastic, we fix the random_state to ensure reproducability
    ica = FastICA(n_components=2, random_state=42) # defaults: max_iter=200, tol=1e-04
    ica_embedding = ica.fit_transform(data_frame)
    A_ = ica.mixing_  # Gets the estimated mixing matrix
    return ica_embedding

In [ ]:
def plot_ica(data_frame, marker_column):
    ica_embedding = train_ica(data_frame)
    return plot_embedding(ica_embedding, data_frame[marker_column], title=f"ICA Embedding: {marker_column}")

In [ ]:
ica_pod_plot = plot_ica(df, "POD")

In [ ]:
ica_pocd_plot = plot_ica(df, "POCD")

#### Try some different ICA embeddings: divide by sex

In [ ]:
ica_pod_males_plot = plot_ica(males, "POD")

In [ ]:
ica_pod_females_plot = plot_ica(females, "POD")

In [ ]:
# save plots to disk
ica_pod_plot.savefig(f"{plots_dir}ICA_Embedding_POD.png")
ica_pocd_plot.savefig(f"{plots_dir}ICA_Embedding_POCD.png")
ica_pod_males_plot.savefig(f"{plots_dir}ICA_Embedding_POD_males.png")
ica_pod_females_plot.savefig(f"{plots_dir}ICA_Embedding_POD_females.png")

### Try some training on PCA/UMAP embeddings

In [ ]:
umapper = umap.UMAP(n_components=10)
pca = PCA(n_components=28)
reducer = pca

pca_train_embedding = reducer.fit_transform(train_data)
pca_test_embedding = reducer.transform(test_data)

In [ ]:
pca.explained_variance_ratio_.sum() 
## 28 components explain 80% variance

### RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(pca_train_embedding, train_pod)
preds_train = clf.predict(pca_train_embedding)
preds = clf.predict(pca_test_embedding)


print((preds_train == train_pod).mean())
print((preds == test_pod).mean())
print(roc_auc_score(test_pod, preds))

In [ ]:
clf = MLPClassifier(batch_size=32, hidden_layer_sizes=(512,))
clf.fit(pca_train_embedding, train_pod)
preds_train = clf.predict(pca_train_embedding)
preds = clf.predict(pca_test_embedding)


print((preds_train == train_pod).mean())
print((preds == test_pod).mean())
print(roc_auc_score(test_pod, preds))

In [ ]:
clf = MLPClassifier(batch_size=32, hidden_layer_sizes=(128,), solver='lbfgs')
clf.fit(data_dict["train_blood"], train_pod)
preds_train = clf.predict(data_dict["train_blood"])
preds = clf.predict(data_dict["test_blood"])


print((preds_train == train_pod).mean())
print((preds == test_pod).mean())
print(roc_auc_score(test_pod, preds))

### Investigate PCA:

In [ ]:
pca = PCA(n_components=None)
pca_train_embedding = pca.fit_transform(train_data)
pca_test_embedding = pca.transform(test_data)
var=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=3)*100)

plt.figure().patch.set_color('white')
plt.plot(var, color='#387387')

plt.ylabel('% Variance Explained')
plt.xlabel('# of Features')
plt.title('PCA Analysis')
plt.ylim(30,100.5)
#plt.style.context('seaborn-whitegrid')

ax = plt.gca()
ax.set_facecolor('white')
ax.spines['left'].set_color('#25404B')
ax.spines['bottom'].set_color('#25404B')
ax.tick_params(axis='x', colors='#25404B')
ax.tick_params(axis='y', colors='#25404B')
ax.yaxis.label.set_color('#25404B')
ax.xaxis.label.set_color('#25404B')
ax.title.set_color('#25404B')
ax.xaxis.set_ticks(np.arange(0, 100, 10))

#print(sorted(pca.explained_variance_ratio_))

In [ ]:
pca_train_embedding

In [ ]:
# Pick the first 28 components because they explain the most variance:
components = pca.components_[:28]

In [ ]:
sns.heatmap(components.T)

In [ ]:
components[np.abs(components) < 0.2] = 0
#components[np.abs(components) > 0.1] = 1

In [ ]:
sns.heatmap(components.T)

In [ ]:
important_features = set()
for comp in components:
    names = data_dict["data_names"][np.abs(comp) > 0.1]
    print(names)
    important_features.update(names)
    print()

In [ ]:
important_features

In [ ]:
data_dict["data_names"][np.abs(components[0]) > 0.1]

In [ ]:
data_dict["data_names"][np.abs(components[1]) > 0.1]

In [ ]:
plt.plot(sorted(components[2]))

## PCA+UMAP

In [ ]:
pca = PCA(n_components=28)
pca_train_embedding = pca.fit_transform(data_dict["train_blood"])
pca_test_embedding = pca.transform(data_dict["test_blood"])

umapper = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.01)
#train_labels = train_data[:, list(static_names).index("MMSE")] # Can be train_pod, None or e.g.: test_data[:, list(static_names).index("MMSE")]
#train_labels = train_labels > np.median(train_labels) # binarize
train_embedding = umapper.fit_transform(pca_train_embedding)
test_embedding = umapper.transform(pca_test_embedding)

In [ ]:
plot_embedding(pca_train_embedding[:,:2], train_pod, "Train Data PCA")

In [ ]:
plot_embedding(train_embedding, train_pod, "Train Data - POD", color_scheme="light")

In [ ]:
plot_embedding(test_embedding, test_pod, "Test Data - POD", color_scheme="light")

In [ ]:
print(data_dict["static_names"])
feature = "MMSE"
labels = test_data[:, list(data_dict["static_names"]).index(feature)]
print("Median: ", np.median(labels))
#print(np.unique(labels))
labels = labels > np.median(labels)
plot_embedding(test_embedding, labels, "Test Data - " + feature, color_scheme="light")

## UMAP

In [ ]:
len(train_data)

In [ ]:
umapper = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.01)
train_labels = train_data[:, list(data_dict["static_names"]).index("MMSE")] # Can be train_pod, None or e.g.: test_data[:, list(static_names).index("MMSE")]
train_labels = train_labels > np.median(train_labels) # binarize

train_embedding = umapper.fit_transform(train_data)
test_embedding = umapper.transform(test_data)

In [ ]:
plot_embedding(train_embedding, train_pod, "Train Data")

In [ ]:
print(data_dict["static_names"])
labels = train_data[:, list(data_dict["static_names"]).index("MMSE")]
print("Median: ", np.median(labels))
#print(np.unique(labels))
labels = labels > np.median(labels)
plot_embedding(train_embedding, labels, "Train Data")

### Test data

In [ ]:
plot_embedding(test_embedding, test_pod, "Test Data")

In [ ]:
print(data_dict["static_names"])
labels = test_data[:, list(data_dict["static_names"]).index("MMSE")]
print("Median: ", np.median(labels))
#print(np.unique(labels))
labels = labels > np.median(labels)
plot_embedding(test_embedding, labels, "Test Data")

In [ ]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(train_embedding, train_pod)
preds = clf.predict(test_embedding)
print((preds == test_pod).mean())
print(roc_auc_score(test_pod, preds))

## Basic RF classification tests:

In [ ]:
weights = class_weight={0: 1 - train_pod.mean(),1: train_pod.mean()}
clf = RandomForestClassifier(n_estimators=1000, class_weight=weights)
clf.fit(train_data, train_pod)

In [ ]:
preds = clf.predict(test_data)
print((preds == test_pod).mean())
print(roc_auc_score(test_pod, preds))

In [ ]:
zipped = zip(clf.feature_importances_, data_dict["data_names"])

In [ ]:
sorted_feats = sorted(zipped, key=lambda x: x[0], reverse=True)

In [ ]:
plt.plot(sorted(clf.feature_importances_))
plt.ylim(0, 0.02)

In [ ]:
sorted_feats[:15]

In [ ]:
sorted_feats[-10:]

In [ ]:
best_feats = [feat[1] for feat in sorted_feats if feat[0] > 0.014]
best_feats

In [ ]:
idcs = [list(data_dict["data_names"]).index(feat) for feat in best_feats]
idcs

In [ ]:
train_pod.shape

In [ ]:
weights = None#class_weight={0: 1 - train_pod.mean(), 1: train_pod.mean()}
clf = RandomForestClassifier(n_estimators=1000, class_weight=weights)
clf.fit(train_data[:, idcs], train_pod)

In [ ]:
train_preds = clf.predict(train_data[:, idcs])
preds = clf.predict(test_data[:, idcs])
print("Train metrics:")
print((train_preds == train_pod).mean())
print("Test metrics:")
print((preds == test_pod).mean())
print(roc_auc_score(test_pod, preds))
print(classification_report(test_pod, preds))

In [ ]:
1 - np.mean(test_pod)

In [ ]:
1 - np.mean(train_pod)

## Under sampled training:

In [ ]:
pod_neg_idcs = np.where(train_pod == 0)[0]
len(pod_neg_idcs)

In [ ]:
pod_pos_idcs = np.where(train_pod == 1)[0]

In [ ]:
rands = np.random.randint(0, len(pod_neg_idcs), len(pod_pos_idcs))

In [ ]:
idcs = pod_neg_idcs[rands]

In [ ]:
all_idcs = list(idcs) + list(pod_pos_idcs)

In [ ]:
balanced_train_data = train_data[all_idcs]

In [ ]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(train_data[all_idcs], train_pod[all_idcs])

In [ ]:
preds = clf.predict(test_data)
print((preds == test_pod).mean())
print(roc_auc_score(test_pod, preds))
print(classification_report(test_pod, preds))